### to do 
- [ ] generates root data from primary source every time
- [ ] bases sorted by none > irreg > caus > pass > intens > desid etc
- [ ] add notes - related to √gaj etc
- [ ] link to fix me fix me
- [ ] import \<b>bold tag from roots sheet
- [ ] √saj (stick, cling) = Pāṇinīya Dhātupāṭha:	sṛj  xxx
- [ ] include dhp dhm groups
- [ ] delete () from Pāṇinīya Dhātupāṭha:	(tark) nan (thinking, reasoning) etc
- [ ] gambhiro lets make this thing!	
- [ ] add lit. meanings
- [ ] pati u √ṭhā = ud = all
- [ ] root group 1 a bhuuvaadigana etc

orange 'color:#F95700'
blue 'color:#00A4CC'

setup

In [1]:
import pandas as pd
from datetime import datetime
start_time = datetime.now()


set up sorter

In [2]:
pāli_alphabet = ["√", " ", "a", "ā", "i", "ī", "u", "ū", "e", "o", "k", "kh", "g", "gh", "ṅ", "c", "ch", "j", "jh", "ñ", "ṭ", "ṭh", "ḍ", "ḍḥ", "ṇ", "t", "th", "d", "dh", "n", "p", "ph", "b", "bh", "m", "y", "r", "l", "s", "v", "h", "ḷ", "ṃ", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "<"]

def tokeniser(word):
    dl = [i for i in pāli_alphabet if len(i) > 1]
    for i in dl:
        word = word.replace(i, '/{}'.format(i))
    wordVe = []
    k = -3
    for j in range(len(word)):
        if word[j] == '/':
            k = j
            wordVe.append(word[j + 1:j + 3])
        if j > k + 2:
            wordVe.append(word[j])
    return wordVe

def sortKey(word):
    pāli_alphabet_string = '-'.join(pāli_alphabet)
    return [pāli_alphabet_string.find('-' + x + '-') for x in tokeniser(word)]


In [3]:
# # generate dpd roots.csv

# from ods_to_csv import convert_dpd_roots_to_csv

# convert_dpd_roots_to_csv()

generate roots sheet csv

In [4]:
roots_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/roots.csv", sep="\t", dtype=str)

roots_df = roots_df[roots_df["Count"] != 0] # remove roots with no examples

roots_df.sort_values(by = ["Root"], inplace=True, ignore_index=True, key=lambda x: x.map(sortKey)) #sort

roots_df["In Comps"].fillna(value="", inplace=True) # fill NA value "In Comps"
roots_df["Dhātupātha"] = roots_df["Dhātupātha"].str.replace("-", "")
roots_df["Dhātupātha"] = roots_df["Dhātupātha"].str.replace(".0", "", regex= False)
roots_df["Kaccāyana Dhātu Mañjūsā"] = roots_df["Kaccāyana Dhātu Mañjūsā"].str.replace("-", "")
roots_df["Kaccāyana Dhātu Mañjūsā"] = roots_df["Kaccāyana Dhātu Mañjūsā"].str.replace(".0", "", regex= False)
roots_df["Cl"] = roots_df["Cl"].str.replace(".0", "", regex= False)
# roots_df = roots_df.astype({"Count": int, "Group":int})

# print (roots_df.columns)
# print (roots_df)



extract bases

In [5]:
start_extract_bases = datetime.now()

bases_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd.csv", sep="\t", dtype=str)
bases_df.fillna("", inplace=True) 

roots_df_count = roots_df.shape[0]
row = 0

for row in range(roots_df_count):
	root = roots_df.iloc[row, 2]
	root_group = roots_df.iloc[row, 5]
	root_meaning = roots_df.iloc[row, 8]

	test1 = bases_df["Pāli Root"] == root
	test2 = bases_df["Grp"] == root_group
	test3 = bases_df["Root Meaning"] == root_meaning
	test4 = bases_df["Base"].str.contains(fr">")

	filter = test1 & test2 & test3 & test4
	bases_filtered = bases_df.loc[filter, ["Base"]]
	bases_filtered.drop_duplicates("Base", inplace=True, keep="first")
	bases_filtered = bases_filtered.dropna()
	bases_filtered = bases_filtered.sort_values(by="Base", key=lambda x: x.str.len())
	bases_filtered = bases_filtered["Base"].str.replace("^.+ > ", "")

	bases_filtered_size = bases_filtered.shape[0]

	with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/bases/{root} {root_group} {root_meaning}.csv", "w") as output_file:
		bases_filtered_size = bases_filtered.shape[0]

		if bases_filtered_size == 0:
			output_file.write(f"-")
		if bases_filtered_size > 0:
			bases_filtered.to_csv(output_file, header=False, index=False, sep="\t")

end_extract_bases = datetime.now()

/tmp/ipykernel_11389/338649195.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  bases_filtered = bases_filtered["Base"].str.replace("^.+ > ", "")


extract families

In [6]:
start_extract_families = datetime.now()

root_families_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/root families for dictionary.csv", sep="\t")

root_families_df.sort_values(by = ["Family"], inplace=True, ignore_index=True, key=lambda x: x.map(sortKey)) #sort

roots_df_count = roots_df.shape[0]
row = 0

for row in range(roots_df_count):
	root = roots_df.iloc[row, 2]
	root_group = roots_df.iloc[row, 5]
	root_meaning = roots_df.iloc[row, 8]

	test1 = root_families_df["Root"] == root
	test2 = root_families_df["Meaning"] == root_meaning
	test3 = root_families_df["Grp"] == float(root_group)
	root_families_filtered = root_families_df.loc[test1 & test2 & test3, ["Family"]]

	root_families_filtered.drop_duplicates("Family", inplace=True, keep='first')

	with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/families/{root} {root_group} {root_meaning}.csv", "w") as output_file:
		root_families_filtered.to_csv(output_file, header=False, index=False, sep="\t")

end_extract_families = datetime.now()

generate df for each family

In [7]:
start_generate_fam_df = datetime.now()

row = 0



for row in range(roots_df_count): # cycle though each root
	root = roots_df.iloc[row, 2]
	root_group = roots_df.iloc[row, 5]
	root_meaning = roots_df.iloc[row, 8]
	# print("")
	# print(f"{root = }")
	# print(f"{root_group = }")
	# print(f"{root_meaning = }")

	try:
		family_df = pd.read_csv(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/families/{root} {root_group} {root_meaning}.csv", sep="\t", header=None)

	except:
		error_log = open("errorlog.txt", "a") 
		error_log.write(f"{root} {root_group} {root_meaning}\n")
		error_log.close

	family_df_length = family_df.shape[0]

	row = 0
	for row in range(family_df_length):
		subfamily = family_df.iloc[row, 0]

		test1 = root_families_df["Root"] == root
		test2 = root_families_df["Meaning"] == root_meaning
		test3 = root_families_df["Grp"] == root_group
		test4 = root_families_df["Family"] == subfamily
		filter = test1 & test2 & test3 & test4
		root_families_filtered = root_families_df.loc[filter, ["Pāli", "POS", "English"]]

		root_families_filtered.sort_values(by = ["Pāli"], inplace=True, ignore_index=True, key=lambda x: x.map(sortKey)) #sort

		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/subfamilies/{root} {root_group} {root_meaning} {subfamily}.csv", "w") as output_file:
			root_families_filtered.to_csv(output_file, sep="\t", header=False, index=False)

end_generate_fam_df = datetime.now()

write root information

In [8]:
start_write_html = datetime.now()

roots_df_count = roots_df.shape[0]

import re

for row in range(roots_df_count):
	root = roots_df.iloc[row, 2]
	root_in_comps = roots_df.iloc[row, 3]
	root_has_verb = roots_df.iloc[row, 4]
	root_group = roots_df.iloc[row, 5]
	root_sign = roots_df.iloc[row, 6]
	root_meaning = roots_df.iloc[row, 8]
	base_file = open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/bases/{root} {root_group} {root_meaning}.csv")
	base = base_file.read()
	base = re.sub("\n", " ", base)

	sk_root = roots_df.iloc[row, 9]
	sk_root_meaning = roots_df.iloc[row, 10]
	sk_root_class = roots_df.iloc[row, 11]
	example = roots_df.iloc[row, 12]
	dhp_no = roots_df.iloc[row, 13]
	dhp_root = roots_df.iloc[row, 14]
	dhp_meaning = roots_df.iloc[row, 15]
	dhp_english = roots_df.iloc[row, 16]
	dhm_no = roots_df.iloc[row, 17]
	dhm_root = roots_df.iloc[row, 18]
	dhm_meaning = roots_df.iloc[row, 19]
	dhm_english = roots_df.iloc[row, 20]
	sdn_root = roots_df.iloc[row, 21]
	sdn_meaning = roots_df.iloc[row, 22]
	sdn_english = roots_df.iloc[row, 23]
	pdp_root = roots_df.iloc[row, 24]
	pdp_meaning = roots_df.iloc[row, 25]
	pdp_english = roots_df.iloc[row, 26]
	notes = roots_df.iloc[row, 27]

	if str(root_group) == "1.0":
		root_group_pali = "bhūvādigaṇa"
	if str(root_group) == "2.0":
		root_group_pali = "rudhādigaṇa"
	if str(root_group) == "3.0":
		root_group_pali = "divādigaṇa"
	if str(root_group) == "4.0":
		root_group_pali = "svādigaṇa"
	if str(root_group) == "5.0":
		root_group_pali = "kiyādigaṇa"
	if str(root_group) == "6.0":
		root_group_pali = "gahādigaṇa"
	if str(root_group) == "7.0":
		root_group_pali = "tanādigaṇa"
	if str(root_group) == "8.0":
		root_group_pali = "curādigaṇa"

	html_style = """<style>
	{  font-family: "Verajja Serif", "DejaVu Sans", sans-serif;}
	td {vertical-align: top;}
	</style>
	"""

	html_heading =  f"""
	<h3>{root} ({root_meaning})</h3>
	"""

	html_table1 = f"""
	<table><tbody>
	<tr><td style = 'color:#F95700'>Pāli Root:</td><td>{root} {root_group} {root_sign} ({root_meaning})</td></tr>
	<tr><td style = 'color:#F95700'>Base(s):</td><td>{base}</td></tr>	
	"""

	# Root in comps
	if root_in_comps != "":
		html_table2 = f"""
		<tr><td style = 'color:#F95700'>In Compounds:</td><td>{root_in_comps}</td></tr>	
		"""
	else:
		html_table2 = f""""""

	# Dhātupātha
	if dhp_root != "-":
		html_table4 = f"""
		<tr><td style = 'color:#F95700'>Dhātupātha:</td><td>{dhp_root} <i>{dhp_meaning}</i> ({dhp_english}) <i>{dhp_no}</i> </td></tr>	
		"""
	else:
		html_table4 = f"""
		<tr><td style = 'color:#F95700'>Dhātupātha:</td><td>-</td></tr>	
		"""

	# Dhātumañjūsa
	if dhm_root != "-":
		html_table5 = f"""
		<tr><td style = 'color:#F95700'>Dhātumañjūsa:</td><td>{dhm_root} <i>{dhm_meaning}</i> ({dhm_english}) <i>{dhm_no}</i> </td></tr>	
		"""
	else:
		html_table5 = f"""
		<tr><td style = 'color:#F95700'>Dhātumañjūsa:</td><td>-</td></tr>
		"""

	# Saddanīti
	if sdn_root != "-":
		html_table6 = f"""
		<tr><td style = 'color:#F95700'>Saddanīti:</td><td>{sdn_root} <i>{sdn_meaning}</i> ({sdn_english})</td></tr>	
		"""
	else:
		html_table6 = f"""
		<tr><td style = 'color:#F95700'>Saddanīti:</td><td>-</td></tr>
		"""

	# Sanskrit
	html_table3 = f"""
	<tr><td style = 'color:#F95700'>Sanskrit Root:</td><td style = 'color:gray'>{sk_root} {sk_root_class} ({sk_root_meaning})</td></tr>	
	"""

	# Pāṇinīya Dhātupāṭha
	if pdp_root != "-":
		html_table7 = f"""
		<tr><td style = 'color:#F95700'>Pāṇinīya Dhātupāṭha:</td><td style = 'color:gray'>{pdp_root} <i>{pdp_meaning}</i> ({pdp_english})</td></tr>	
		"""
	else:
		html_table7 = f"""
		<tr><td style = 'color:#F95700'>Pāṇinīya Dhātupāṭha:</td><td>-</td></tr>	
		"""

	html_table8 = f"""
	</tbody></table>
	"""

	with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/html output/{root} {root_group} {root_meaning}.html", "w") as output_file:
		output_file.write(html_style)
		output_file.write(html_heading)
		output_file.write(html_table1)
		output_file.write(html_table2)
		output_file.write(html_table4)
		output_file.write(html_table5)
		output_file.write(html_table6)
		output_file.write(html_table3)
		output_file.write(html_table7)

		output_file.write(html_table8)
		output_file.write("<hr>")

end_write_html = datetime.now()

generate html for each root family

In [9]:
start_generate_fam_html = datetime.now()

row = 0
for row in range(roots_df_count):
	root = roots_df.iloc[row, 2]
	root_group = int(roots_df.iloc[row, 5])
	root_meaning = roots_df.iloc[row, 8]

	family_df = pd.read_csv(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/families/{root} {root_group} {root_meaning}.csv", sep="\t", header=None)

	family_df_length = family_df.shape[0]

	with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/html output/{root} {root_group} {root_meaning}.html", "a") as output_file:
		output_file.write(f"<h4>Root Family (all words with the same prefix + root)</h4>")
		output_file.write(f"<table><tbody>")
		# output_file.write(f"<tr><td style = 'color:#00A4CC'>Pāli word</td><td style = 'color:gray'>(pos)</td><td style = 'color:black'>English meaning</td>")


		for row in range(family_df_length):
			subfamily = family_df.iloc[row, 0]

			subfamily_df = pd.read_csv(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/subfamilies/{root} {root_group} {root_meaning} {subfamily}.csv", sep="\t", header=None)	
			
			subfamily_df_length = subfamily_df.shape[0]

			for row_sf in range(subfamily_df_length):
				sf_pali = subfamily_df.iloc[row_sf, 0]
				sf_pos = subfamily_df.iloc[row_sf, 1]
				sf_english = subfamily_df.iloc[row_sf, 2]
				# print(f"{row_sf}\t{sf_pali}\t({sf_pos})  \t{sf_english}")

				if row_sf == 0 and row_sf == subfamily_df_length -1:
					output_file.write(f"<tr><th style='text-align:left'>{subfamily}</th></tr>")
					output_file.write(f"<tr><td style = 'color:#00A4CC'>{sf_pali}</td>")
					output_file.write(f"<td style = 'color:gray'>({sf_pos})  </td>")
					output_file.write(f"<td>{sf_english}</td></tr>")
					output_file.write(f"<tr><td></td></tr>")

				if row_sf == 0 and row_sf < subfamily_df_length -1:
					output_file.write(f"<tr><th style='text-align:left'>{subfamily}</th></tr>")
					output_file.write(f"<tr><td style = 'color:#00A4CC'>{sf_pali}</td>")
					output_file.write(f"<td style = 'color:gray'>({sf_pos})  </td>")
					output_file.write(f"<td>{sf_english}</td></tr>")

				elif row_sf > 0 and row_sf < subfamily_df_length -1:
					output_file.write(f"<tr><td style = 'color:#00A4CC'>{sf_pali}</td>")
					output_file.write(f"<td style = 'color:gray'>({sf_pos})  </td>")
					output_file.write(f"<td>{sf_english}</td></tr>")

				elif row_sf > 0 and row_sf == subfamily_df_length -1:
					output_file.write(f"<tr><td style = 'color:#00A4CC'>{sf_pali}</td>")
					output_file.write(f"<td style = 'color:gray'>({sf_pos})  </td>")
					output_file.write(f"<td >{sf_english}</td>")
					output_file.write(f"<tr><td></td></tr>")

end_generate_fam_html = datetime.now()

EmptyDataError: No columns to parse from file

generate html for each root subfamily

In [10]:
start_generate_subfam_html = datetime.now()

row = 0
for row in range(roots_df_count):
	root = roots_df.iloc[row, 2]
	root_group = roots_df.iloc[row, 5]
	root_meaning = roots_df.iloc[row, 8]

	family_df = pd.read_csv(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/families/{root} {root_group} {root_meaning}.csv", sep="\t", header=None)

	family_df_length = family_df.shape[0]

	for row in range(family_df_length):
		subfamily = family_df.iloc[row, 0]

		subfamily_df = pd.read_csv(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/subfamilies/{root} {root_group} {root_meaning} {subfamily}.csv", sep="\t", header=None)	

		subfamily_df_length = subfamily_df.shape[0]

		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/subfamily html/{root} {root_group} {root_meaning} {subfamily}.html", "w") as output_file:
			output_file.write(f"""<table class= "table3"><tbody>""")

			for row_sf in range(subfamily_df_length):
				sf_pali = subfamily_df.iloc[row_sf, 0]
				sf_pos = subfamily_df.iloc[row_sf, 1]
				sf_english = subfamily_df.iloc[row_sf, 2]
				# print(f"{row_sf}\t{sf_pali}\t({sf_pos})  \t{sf_english}")

				output_file.write(f"<tr><th>{sf_pali}</th>")
				output_file.write(f"<td>{sf_pos}</td>")
				output_file.write(f"<td>{sf_english}</td></tr>")
			output_file.write(f"""</tbody></table>""")
				
end_generate_subfam_html = datetime.now()

EmptyDataError: No columns to parse from file

timer

In [ ]:
end_time = datetime.now()
print('{} extract bases'.format(end_extract_bases - start_extract_bases))
print('{} extract families'.format(end_extract_families - start_extract_families))
print('{} generate fam dataframes'.format(end_generate_fam_df - start_generate_fam_df))
print('{} write root html'.format(end_write_html - start_write_html))
print('{} write family html'.format(end_generate_fam_html - start_generate_fam_html))
print('{} write family html'.format(end_generate_subfam_html - start_generate_subfam_html))
print('{} total'.format(end_time - start_time))

0:00:14.967084 extract bases
0:00:02.699278 extract families
0:00:13.504120 generate fam dataframes
0:00:00.465536 write root html
0:00:04.370303 write family html
0:00:04.671060 write family html
0:00:40.921611 total
